In [7]:
# from transformers import AutoFeatureExtractor, ResNetForImageClassification
# import torch
# from datasets import load_dataset

# dataset = load_dataset("huggingface/cats-image")
# image = dataset["test"]["image"][0]

# feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-101")
# model = ResNetForImageClassification.from_pretrained("microsoft/resnet-101")

# inputs = feature_extractor(image, return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# # model predicts one of the 1000 ImageNet classes
# predicted_label = logits.argmax(-1).item()
# print(model.config.id2label[predicted_label])


In [8]:
# image

In [9]:
# model

# 예시1

In [10]:
# import torch
# import torchvision.models as models
# import torchvision.transforms as transforms

# # 1. Load a pre-trained ResNet-101 model
# # You can choose to use the default ImageNet1K_V2 weights for improved accuracy
# # or IMAGENET1K_V1 for the original paper's weights.
# model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)

# # Set the model to evaluation mode (important for inference)
# model.eval()

# # 2. Define image transformations for inference
# # These transformations match the preprocessing used for the pre-trained ImageNet weights.
# preprocess = transforms.Compose([
#     transforms.Resize(256),            # Resize the image to 256x256
#     transforms.CenterCrop(224),        # Crop the center 224x224
#     transforms.ToTensor(),             # Convert to PyTorch Tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize
# ])

# # 3. Prepare an example image (replace with your own image loading)
# # For demonstration, we'll create a dummy image.
# # In a real scenario, you would load an image using PIL: Image.open("your_image.jpg")
# dummy_image = Image.new('RGB', (256, 256), color = 'red')

# # Apply transformations to the image
# input_tensor = preprocess(dummy_image)
# input_batch = input_tensor.unsqueeze(0) # Add a batch dimension

# # 4. Perform inference
# with torch.no_grad(): # Disable gradient calculations for inference
#     output = model(input_batch)

# # 5. Process the output (e.g., get top predictions)
# # The output will be log-probabilities for 1000 ImageNet classes.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# top5_prob, top5_catid = torch.topk(probabilities, 5)

# print("Top 5 predicted categories and their probabilities:")
# for i in range(top5_prob.size(0)):
#     print(f"  Category ID: {top5_catid[i].item()}, Probability: {top5_prob[i].item():.4f}")


# 예시2

In [11]:
import os, math, time
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
import random

NUM_CLASSES = 15 # 분류할 클래스 수
IMG_SIZE = 224 # 입력 이미지의 해상도(256x256 픽셀), 마지막 10~20ep에 320으로 리핏 권장
BATCH = 48 # 배치 크기
EPOCHS = 3 # 에폭 수
BASE_LR = 5e-4 # 기본 learing rate(0.0005)
WD = 0.05 # 가중치 감쇠(weight decay)(0.05) - 과적화 방지를 위한 정규화 기법
WARMUP_EPOCHS = 1 # 워밍업 에폭 수 - 학습률을 점진적으로 증가시크는 에폭 수, 학습 초기에 안정적인 학습을 위한 warmup 기법
SEED = 42

# TRAIN_DATA_DIR = "/content/drive/MyDrive/서울대 교육과정/딥러닝_피부데이터_프로젝트/data/training/01.원천데이터"   # 훈련 데이터 경로
# VAL_DATA_DIR = "/content/drive/MyDrive/서울대 교육과정/딥러닝_피부데이터_프로젝트/data/validation/01.원천데이터"   # 검증 데이터 경로
TRAIN_DATA_DIR = "./data/training/01.원천데이터"   # 훈련 데이터 경로
VAL_DATA_DIR = "./data/validation/01.원천데이터"   # 검증 데이터 경로

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# Set random seeds for reproducibility
def set_seed(seed=SEED):
    """모든 랜덤 시드를 설정하는 함수"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    print('set_seed', seed)
    # torch.cuda.manual_seed_all(seed)  # 멀티 GPU 사용 시
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    # os.environ['PYTHONHASHSEED'] = str(seed)

def worker_init_fn(worker_id):
    """각 워커 프로세스에서 동일한 시드 설정"""
    worker_seed = torch.initial_seed() + worker_id
    set_seed(SEED)

# 시드 설정
set_seed(SEED)


# Check PyTorch version and CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")

# Set device for computations
device = torch.device(DEVICE)
print(f"Using device: {device}")

# torch.set_default_device(device)

set_seed 42
PyTorch version: 2.8.0
CUDA available: False
CUDA device count: 0
Using device: cpu


In [12]:
# 1) 데이터 변환(색 왜곡은 과하지 않게)
mean = [0.485, 0.456, 0.406]   # ImageNet
std  = [0.229, 0.224, 0.225]

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    # transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_dataset = datasets.ImageFolder(TRAIN_DATA_DIR, transform=train_tf)
val_datatset = datasets.ImageFolder(VAL_DATA_DIR, transform=val_tf)
train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=0, generator=torch.Generator(device=DEVICE), worker_init_fn=worker_init_fn,)
val_loader   = DataLoader(val_datatset, batch_size=BATCH, shuffle=False, num_workers=0, generator=torch.Generator(device=DEVICE))

In [13]:
# 2) 모델: ResNet101 + 헤드 교체
weights = models.ResNet101_Weights.IMAGENET1K_V2
model = models.resnet101(weights=weights)
in_feats = model.fc.in_features
model.fc = nn.Linear(in_feats, NUM_CLASSES)
model = model.to(device)

# (옵션) 초반 5~10 에폭 백본 동결 후 해제
FREEZE_WARMUP_EPOCHS = 5
for p in model.parameters():
    p.requires_grad = True
for p in list(model.children())[:-1]:     # fc 제외 백본
    for q in p.parameters():
        q.requires_grad = False

In [14]:

# 3) 손실/옵티마/스케줄러
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=BASE_LR, weight_decay=WD)

def lr_lambda(current_epoch):
    if current_epoch < WARMUP_EPOCHS:
        return float(current_epoch + 1) / WARMUP_EPOCHS
    # cosine
    t = (current_epoch - WARMUP_EPOCHS) / max(1, (EPOCHS - WARMUP_EPOCHS))
    return 0.5 * (1 + math.cos(math.pi * t))

# 스케쥴러
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# 스케일러
scaler = torch.amp.GradScaler(DEVICE)


In [15]:
# 평가 함수
def evaluate(model, loader):
    model.eval()
    loss_sum, n = 0.0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            with torch.amp.autocast(device_type=DEVICE, dtype=torch.float16):
                logits = model(x)
                loss = criterion(logits, y)
            loss_sum += loss.item() * x.size(0); n += x.size(0)
            y_true.extend(y.detach().cpu().tolist())
            y_pred.extend(torch.argmax(logits, dim=1).detach().cpu().tolist())
    f1 = f1_score(y_true, y_pred, average="macro")
    return loss_sum / n, f1

best_f1, best_path = 0.0, "resnet101_best.pth"

In [16]:
for epoch in range(EPOCHS):
    t0 = time.time()
    model.train()
    # (에폭 전환 시점에 동결 해제)
    if epoch == FREEZE_WARMUP_EPOCHS:
        for p in model.parameters():
            p.requires_grad = True
        optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WD)
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    train_loss, n = 0.0, 0
    print(f"Epoch {epoch+1}/{EPOCHS} - Training...")

    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)

        with torch.amp.autocast(device_type=DEVICE):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item() * x.size(0)
        n += x.size(0)
        
        # 진행 상황 출력 (매 10배치마다)
        if (batch_idx + 1) % 10 == 0:
            print(f"  Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    scheduler.step()
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Evaluating...")
    val_loss, val_f1 = evaluate(model, val_loader)

    print(f"[{epoch+1:03d}/{EPOCHS}] "
          f"train_loss={train_loss/n:.4f} val_loss={val_loss:.4f} val_macroF1={val_f1:.4f} "
          f"lr={scheduler.get_last_lr()[0]:.2e} time={time.time()-t0:.1f}s")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save({"model": model.state_dict(), "epoch": epoch+1, "f1": best_f1}, best_path)
        print(f"New best F1: {best_f1:.4f}, saved to {best_path}")

print("Training completed!")
print(f"Best F1: {best_f1:.4f}, saved to {best_path}")


Epoch 1/3 - Training...
  Batch 10/250, Loss: 2.4955
  Batch 20/250, Loss: 2.2827
  Batch 30/250, Loss: 2.0888
  Batch 40/250, Loss: 1.8946
  Batch 50/250, Loss: 1.8268
  Batch 60/250, Loss: 1.7670
  Batch 70/250, Loss: 1.5565
  Batch 80/250, Loss: 1.4967
  Batch 90/250, Loss: 1.4369
  Batch 100/250, Loss: 1.3676
  Batch 110/250, Loss: 1.3843
  Batch 120/250, Loss: 1.2563
  Batch 130/250, Loss: 1.2416
  Batch 140/250, Loss: 1.1141
  Batch 150/250, Loss: 1.1054
  Batch 160/250, Loss: 0.9457
  Batch 170/250, Loss: 1.2250
  Batch 180/250, Loss: 0.9338
  Batch 190/250, Loss: 1.1272
  Batch 200/250, Loss: 1.0451
  Batch 210/250, Loss: 1.0058
  Batch 220/250, Loss: 0.9245
  Batch 230/250, Loss: 0.8964
  Batch 240/250, Loss: 0.9835
  Batch 250/250, Loss: 1.0173
Epoch 1/3 - Evaluating...
[001/3] train_loss=1.3947 val_loss=0.8655 val_macroF1=0.9466 lr=5.00e-04 time=9830.4s
New best F1: 0.9466, saved to resnet101_best.pth
Epoch 2/3 - Training...
  Batch 10/250, Loss: 0.9754
  Batch 20/250, Loss: